In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Modularized Functions ###

# function for converting minutes:seconds to decimal minutes
def convert_minutes(list):
    if len(list) > 1:
        return round(int(list[0]) + (int(list[1])/60), 3)
    else:
        return 0

In [29]:
# monthList = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']
monthList = ['january']

boxscores = []
for month in monthList:
    
    print('loading games from ' + month + '.')
    
    # accessing a table containing game info from the month of october
    html_month = requests.get('https://www.basketball-reference.com/leagues/NBA_2019_games-' + month + '.html').content
    soup_month = BeautifulSoup(html_month, features='lxml')

    # extracting links to all of the boxscores from these games
    games = []
    for link in soup_month.find_all('a'):
        gameLink = link.get('href')
        if ('boxscore' in gameLink) and ('month' not in gameLink) and (len(gameLink) > 11):
            games.append(gameLink)

    # accessing each boxscore and saving it
    for i in range(len(games)):

        # extracting game information from url
        gameID = games[i].replace('/boxscores/', '').replace('.html', '')
        date = gameID[0:4] + "-" + gameID[4:6] + "-" + gameID[6:-4]
        location = gameID[-3:]
        
        print('   ' + gameID + ' is being loaded.')

        # accessing the first boxscore of month
        html_game = requests.get('https://www.basketball-reference.com/' + games[i]).content
        soup_game = BeautifulSoup(html_game, features='lxml')

        # scrapes basic box score for both teams, converts to DataFrame
        for t in soup_game.findAll('table', id = lambda x: 'game-basic' in x):
            team = t.get('id').replace('box-', '').replace('-game-basic','') # scraping team abbreviations
            newDF = pd.read_html(str(t), index_col = 0, header=1)[0] # converting html table to DataFrame

            del newDF.index.name
            newDF = newDF[newDF.index != 'Team Totals']
            newDF = newDF[newDF.index != 'Reserves']

            # adding team abbreviations to boxscore
            team_array = [team for i in range(len(newDF.index))]
            date_array = [date for i in range(len(newDF.index))]
            location_array = [location for i in range(len(newDF.index))]
            gameID_array = [gameID for i in range(len(newDF.index))]
            newDF.insert(0, 'TEAM', team_array)
            newDF.insert(1, 'DATE', date_array)
            newDF.insert(2, 'LOC', location_array)
            newDF.insert(3, 'GAME_ID', gameID_array)

            # standardizing the date and minutes-played formats
            newDF['DATE'] = pd.to_datetime(newDF['DATE'])
            newDF['MP'] = newDF['MP'].str.split(":").apply(convert_minutes)

            boxscores.append(newDF)
    print('   all games from ' + month + ' have been loaded.')
        
allBoxscores = pd.concat(boxscores).reset_index()

loading games from january.
   201901010TOR is being loaded.
   201901010MIL is being loaded.
   201901010DEN is being loaded.
   201901010SAC is being loaded.
   201901010LAC is being loaded.
   201901020CHO is being loaded.
   201901020CLE is being loaded.
   201901020WAS is being loaded.
   201901020BRK is being loaded.
   201901020BOS is being loaded.
   201901020CHI is being loaded.
   201901020MEM is being loaded.
   201901020PHO is being loaded.
   201901020LAL is being loaded.
   201901030SAS is being loaded.
   201901030SAC is being loaded.
   201901030GSW is being loaded.
   201901040CLE is being loaded.
   201901040BOS is being loaded.
   201901040CHI is being loaded.
   201901040MEM is being loaded.
   201901040MIA is being loaded.
   201901040MIN is being loaded.
   201901040MIL is being loaded.
   201901040PHO is being loaded.
   201901040LAL is being loaded.
   201901040POR is being loaded.
   201901050DEN is being loaded.
   201901050DET is being loaded.
   201901050PHI

NameError: name 'df' is not defined

In [58]:
allBoxscores = allBoxscores.rename(columns = {'PLAYERS':'PLAYER'})

points = allBoxscores[['PLAYER','PTS']]

# allBoxscores.dtypes


points.groupby(['PLAYER'])['PTS'].mean()

DataError: No numeric types to aggregate